# About the Project

Product prices vary quite frequently on Amazon. During sales the prices decrease drastically compared to their trending prices. 
The project seeks to enable a user to track product details of a product listed on amazon. User can provide a file with a url and the desired price at which they want to buy the given product. When the price reaches or decreases beyond your desired price, the program sets off an alarm indicating that the product can be bought now.

The code also provides an output file with metadata of the product in the url present. 

### A. Importing Packages

In [1]:
import bs4 as bs
from bs4 import BeautifulSoup
import sys
import schedule
import time
import requests
import urllib.request
import datetime
import pandas as pd
import winsound

### B. Defining static variables and importing url files

In [2]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
frequency = 3000  # Set Frequency To 3000 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second

df_files_with_url = pd.read_csv('TRACKER_PRODUCTS.csv')
df_files_with_url.dropna(inplace = True)

### C. Defining function to scrape amazon website

In [3]:
def mainprogram(headers, url, target_price):   
    headers = headers

    r = requests.get(url, headers = headers)
    content = r.content
    
    soup = BeautifulSoup(content)
    
    #Extracting details from the url page
    #Find Title 
    js_test = soup.find('title')
    product_name = js_test.text
    print(product_name)
    #Current Price
    js_money = soup.find(class_ = "a-price-whole")
    current_price = float(js_money.text.replace(",", ""))
    #Number of reviews
    js_reviews = soup.find('span', id = "acrCustomerReviewText").text.split(" ")[0]
    reviews = int(js_reviews.replace(",", ""))
    #Average Ratings
    js_ratings = soup.find(class_ = "a-icon-alt").text.split(" ")[0]
    average_ratings = float(js_ratings)
    #MRP
    js_mrp = soup.find(class_ = "a-size-small a-color-secondary aok-align-center basisPrice")
    js_mrp = soup.find(class_ = "a-price a-text-price").text.split("₹")[1]
    js_mrp = js_mrp.replace(",", "")
    mrp = float(js_mrp)
    #In Stock Status
    js_stock = soup.find(class_ = "a-size-medium a-color-success").text.strip(" ")
    #Seller
    js_seller = soup.find(class_ = "a-size-base a-color-secondary a-nowrap").text
    #query_time
    js_datetime = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    #Query Date
    js_date = datetime.datetime.today().strftime("%d/%m/%Y")
    
    data_dictionary_list = {'Product': [product_name], 'Price':[current_price],
                      "MRP": [mrp],
                      "No. of Reviews": [reviews], 
                      "Average Ratings": [average_ratings], 
                      
                      "Stock Availability Status":[js_stock], 
                      "Seller":[js_seller],
                      "Time of Query": [js_datetime], 
                      "Date of Query": [js_date]}
    
    df = pd.DataFrame(data_dictionary_list)
    current_price = float(current_price)
    your_price = target_price
    if current_price < your_price :
        print("Price decreased book now")
        winsound.Beep(frequency, duration)
    else:
        print("Price is high please wait for the best deal")
    #time.sleep(10)
    return df

### D. Defining the job to be executed

In [4]:
def job():
    df_master = pd.DataFrame()
    print("Tracking....")   
    for i, row in df_files_with_url.iterrows():
        df = mainprogram(headers, row['url'], row['Target Price'])
        df_master = pd.concat([df_master, df], sort = False)
    return df_master

### E. Getting the product features

In [5]:
df_all_price_history = job()
df_all_price_history.to_excel('Product_Details_Amazon.xlsx', index = False)

Tracking....
Sony LinkBuds WF-L900 Truly Wireless Bluetooth Earbuds with Open-Ring Design for Ambient Sound, 17.5 Hrs Battery, DSEE, IPX4, 360RA, Swift Pair & True Wireless Earbuds with Alexa Built-in, Gray : Amazon.in: Electronics
Price is high please wait for the best deal
Sony WF-C500 Bluetooth Truly Wireless in Ear Earbuds with 20 Hrs Battery, True Wireless Earbuds with Mic for Phone Calls, Fast Pair | Instant Bank Discount of INR 1000 on Select Prepaid transactions : Amazon.in: Electronics
Price decreased book now
Sony WI-C100 Wireless Headphones with 25 Hrs Battery, Quick Charge, DSEE-Upscale, Splash Proof (IPX4), 360RA, Swift Pair, Fast Pair, in-Ear Bluetooth Headset with mic for Phone Calls & Music (Black) : Amazon.in: Electronics
Price decreased book now


### F. Auto Schedule price check

In [6]:
schedule.every(30).minutes.do(job)
  
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 

### G. Scheduling the run every day

In [ ]:
schedule.every().day.at("18:45").do(job,'It is 18:45')

while True:
    schedule.run_pending()
    time.sleep(60)